In [ ]:
import math
import numpy as np
import exerciser
import exercise1

class PID:
    def __init__(self, kp: float, kd: float, ki: float):
        self.kp = kp
        self.kd = kd
        self.ki = ki
        self.last_x = math.nan
        self.integral = 0

    def control(self, delta: float, x: float) -> float:
        vx = (x - self.last_x) / delta
        self.last_x = x
        self.integral = min(max(self.integral + x * delta, -100), 100) # TODO: Better anti-windup
        exercise1.plot_value("integral", self.integral)
        return -x * self.kp - vx * self.kd - self.integral * self.ki

# TODO: Move this into exercise1?
def simulate(pid: PID, max_time: int, exercise):
    ex = exercise1.BlockExercise(pid, exercise)
    stable_time = 0
    for _ in range(max_time * exerciser.TPS):
        ex.tick(exerciser.DELTA)
        if abs(ex.vx) <= 0.01 and abs(ex.x) <= 0.01:
            stable_time += exerciser.DELTA
            if stable_time > 1:
                return ex.t
        else:
            stable_time = 0
    return math.inf

In [ ]:
import matplotlib.pyplot as plt

x_axis = np.arange(0, 10, 0.05)
plt.plot(x_axis, [simulate(PID(10, 5, x), max_time=30, exercise=2) for x in x_axis])
plt.ylim(bottom=0);

In [ ]:
import scipy.optimize

def optimize(exercise):
    func = lambda x: simulate(PID(*x), max_time=30, exercise=exercise)
    bounds = [(0, 20), (0, 20), (0, 20)]
    return scipy.optimize.differential_evolution(func, bounds=bounds)

result = optimize(exercise=2)
result

In [ ]:
import exercise1

exercise1.visualize(PID(20, 10, 10), exercise=3)